# Fundamentals of Software Systems (FSS)
**Software Evolution – Part 02 Assignment**

## Submission Guidelines

To correctly complete this assignment you must:
* Carry out the assignment in a team of 2 to 4 students.
* Carry out the assignment with your team only. You are allowed to discuss solutions with other teams, but each team should come up its own personal solution. A strict plagiarism policy is going to be applied to all the artifacts submitted for evaluation.
* As your submission, upload the filled Jupyter Notebook (including outputs) together with the d3 visualization web pages (i.e. upload everything you downloaded including the filled Jupyter Notebook plus your `output.json`)
* The files must be uploaded to OLAT as a single ZIP (`.zip`) file by Dec 15, 2025 @ 23:55.


## Group Members
* Lucia, TODO, TODO
* Oliver, Strassmann, 15-932-726

## Task Context

In this assigment we will be analyzing the _elasticsearch_ project. All following tasks should be done with the subset of commits from tag `v1.0.0` to tag `v1.1.0`.

In [4]:
import pydriller
import subprocess

from enum import Enum
from pathlib import Path


class Modification(Enum):
    ADDED = "Lines added"
    REMOVED = "Lines removed"
    TOTAL = "Lines added + lines removed"
    DIFF = "Lines added - lines removed"


repo_url = "https://github.com/elastic/elasticsearch"
repo_path = "elasticsearch"

if not Path(repo_path).exists():
    print("Cloning the Elasticsearch repository...")
    subprocess.run(["git", "clone", repo_url], check=True)

repo = pydriller.Repository(repo_path, from_tag="v1.0.0", to_tag="v1.1.0")

## Task 1: Author analysis

In the following, please consider only `java` files.

The first task is to get an overview of the author ownership of the _elasticsearch_ project. In particular, we want to understand who are the main authors in the system between the two considered tags, the authors distribution among files and the files distribution among authors. To this aim, perform the following:
- create a dictionary (or a list of tuples) with the pairs author => number of modified files
- create a dictionary (or a list of tuples) with the pairs file => number of authors who modified the file
- visualize the distribution of authors among files: the visualization should have on the x axis the number of authors per file (from 1 to max), and on the y axis the number of files with the given number of authors (so for example the first bar represent the number of files with single author)
- visualize the distribution of files among authors: the visualization should have on the x axis the number of files per author (from 1 to max), and on the y axis the number of authors that own the given number of files (so for example the first bar represent the minor contributors, i.e., the number of authors who own 1 file)

Comment the two distribution visualizations.



Now, let's look at the following 3 packages in more details:
1. `src/main/java/org/elasticsearch/search`
2. `src/main/java/org/elasticsearch/index`
3. `src/main/java/org/elasticsearch/action`

Create a function that, given the path of a package and a modification type (see class Modification above), returns a dictionary of authors => number, where the number counts the total lines added or removed or added+removed or added-removed (depending on the given Modification parameter), for the given package. To compute the value at the package level, you should aggregate the data per file.

Using the function defined above, visualize the author contributions (lines added + lines removed). The visualization should have the author on the x axis, and the total lines on the y axis. Sort the visualization in decreasing amount of contributions, i.e., the main author should be the first.

Compare the visualization for the 3 packages and comment.

## Task 2: Knowledge loss

We now want to analyze the knowledge loss when the main contributor of the analyzed project would leave. For this we will use the circle packaging layout introduced in the "Code as a Crime Scene" book. This assignment includes the necessary `knowledge_loss.html` file as well as the `d3` folder for all dependencies. You task is to create the `output.json` file according to the specification below. This file can then be visualized with the files provided.

For showing the visualization, once you have the output as `output.json` you should
* make sure to have the `knowledge_loss.html` file in the same folder
* start a local HTTP server in the same folder (e.g. with python `python3 -m http.server`) to serve the html file (necessary for d3 to work)
* open the served `knowledge_loss.html` and look at the visualization

For the package you identify as the worst in terms of knowledge loss, investigate the author contributions using the function defined in the previous exercise and comment how the situation is, e.g. how big the gap between the main author and the second biggest contributor for the selected package is.

### Output Format for Visualization

* `root` is always the root of the tree
* `size` should be the total number of lines of contribution
* `weight` can be set to the same as `size`
* `ownership` should be set to the percentage of contributions from the main author (e.g. 0.98 for 98% if contributions coming from the main author)

```
{
  "name": "root",
  "children": [
    {
      "name": "test",
      "children": [
        {
          "name": "benchmarking",
          "children": [
            {
              "author_color": "red",
              "size": "4005",
              "name": "t6726-patmat-analysis.scala",
              "weight": 1.0,
              "ownership": 0.9,
              "children": []
            },
            {
              "author_color": "red",
              "size": "55",
              "name": "TreeSetIterator.scala",
              "weight": 0.88,
              "ownership": 0.9,
              "children": []
            }
          ]
        }
      ]
    }
  ]
}
```

### JSON Export

For exporting the data to JSON you can use the following snippet:
```
import json

with open("output.json", "w") as file:
    json.dump(tree, file, indent=4)
```

In [9]:
from typing import Optional
import random

from pathlib import Path
from pydantic import BaseModel
from pydriller import Repository, ModifiedFile


class Node(BaseModel):
    name: str
    children: list


class LeafNode(BaseModel):
    name: str
    size: int
    author_color: str
    weight: int
    ownership: float


available_colors = [
    "green",
    "red",
    "black",
    "blue",
    "cyan",
    "grey", 
    "magenta", 
    "mint", 
    "orange", 
    "purple",
    "white", 
    "yellow"
]

test_contributions = [0.5, 0.8, 0.75, 0.9, 0.33, 0.450, 0.99]


def _get_new_color_for_author(data: dict) -> str:
    if next_color := next((color for color in available_colors if color not in data.values()), None):
        return next_color

    print("No available colors left for new author, returning default color...")
    return "red"


def _get_main_author_contributions() -> float:
    # TODO: implement actual ownership calculation
    return random.choice(test_contributions)


def _create_new_leaf_node(file: ModifiedFile, author_name: str, colors_map: dict) -> tuple[LeafNode, dict]:
    total_modified_lines = file.added_lines + file.deleted_lines
    author_color = colors_map.setdefault(author_name, _get_new_color_for_author(colors_map))

    return LeafNode(
        name=file.filename.replace(".java", ""),
        size=total_modified_lines,
        author_color=author_color,
        weight=total_modified_lines,
        ownership=_get_main_author_contributions()
    ), colors_map


def get_knowledge_map_data(git_repo_path: str) -> Node:
    repo_obj = Repository(str(git_repo_path))
    authors_and_colors = {}
    root = Node(name="root", children=[])
    max_commits = 1000

    for commit in repo_obj.traverse_commits():
        if max_commits <= 0:
            break

        if java_files := [f for f in commit.modified_files if f.filename.endswith(".java")]:
            for file in java_files:
                current_node = root

                # add sub-modules as children
                parts = file.new_path.split("/") if file.new_path else file.old_path.split("/")
                for sub_module in parts[:-1]:
                    found_child = next((child for child in current_node.children if child.name == sub_module), None)

                    if not found_child:
                        new_node = Node(name=sub_module, children=[])
                        current_node.children.append(new_node)
                        current_node = new_node
                    else:
                        current_node = found_child

                # TODO: If node already exists sum up total line count
                leaf_node, authors_and_colors = _create_new_leaf_node(file, commit.author.name, authors_and_colors)
                current_node.children.append(leaf_node)

        max_commits -= 1

    return root


result = get_knowledge_map_data(repo_path)

Path("./output.json").open("w").write(result.model_dump_json(indent=2))

5251143

## Task 3: Code Churn Analysis

The third and last task is to analyze the code churn of the _elasticsearch_ project. For this analysis we look at the code churn, meaning the daily change in the total number of lines of the project. Visualize the code churn over time bucketing the data by day. Remember that you'll need to interpolate the data for days when there are no commits. Chose an interpolation strategy and justify it.

Look at the churn trend over time and identify two outliers. For each of them:
- identify if it was caused by a single or multiple commits (since you are bucketing the data by day)
- find the hash of the involved commit(s)
- find the involved files
- look at the actual diff

Based on the above, discuss if the outlier is a false positive or should be a reason for concern.